In [9]:
import pickle
import pandas as pd
from google.cloud import bigquery
from cachetools import cached, TTLCache
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
# Connexion à BigQuery
client = bigquery.Client(project='cdiscountwagon')

# Cache les résultats de chargement de données pour 10 minutes
cache = TTLCache(maxsize=100, ttl=600)

@cached(cache)
def load_data():
    query = """
    SELECT *
    FROM `cdiscountwagon.Datasets.Merges_Sales_avec_categ`
    """
    df = client.query(query).to_dataframe()
    return df

# Chargement des données
df = load_data()

In [4]:
df.head()

,product_id_upper,product_id,order_date,on_operation,on_google_shopping,type,avg_price,indice_avg_price,nb_new_customers,total_customers,...,enddate_op,display_date_gs,impression_gs,acquisition_cost_gs,tracking_day_front,on_front,click_on_front,category,product_id_1,Category_1
0,603344402,603344402,2024-03-13,0,0,1P,181.91,NaN,0,0,...,NaT,NaT,NaN,NaN,NaT,0,NaN,bricolage,603344402,BRICOLAGE
1,603344402,603344402,2024-03-14,0,0,1P,181.91,NaN,0,0,...,NaT,NaT,NaN,NaN,NaT,0,NaN,bricolage,603344402,BRICOLAGE
2,603344402,603344402,2024-03-15,0,0,1P,181.91,NaN,0,0,...,NaT,NaT,NaN,NaN,NaT,0,NaN,bricolage,603344402,BRICOLAGE
3,603344402,603344402,2024-03-16,0,0,1P,99.90,NaN,0,0,...,NaT,NaT,NaN,NaN,NaT,0,NaN,bricolage,603344402,BRICOLAGE
4,603344402,603344402,2024-03-17,0,0,1P,99.90,NaN,0,0,...,NaT,NaT,NaN,NaN,NaT,0,NaN,bricolage,603344402,BRICOLAGE


In [5]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['day_of_week_number'] = df['order_date'].dt.dayofweek
df['week_in_month'] = (df['order_date'].dt.day - 1) // 7 + 1
df['avg_price'] = df['avg_price'].round(2)
avg_prix_mean = df['indice_avg_price'].mean()
df['indice_avg_price'].fillna(avg_prix_mean, inplace=True)
df['impression_gs'].fillna(0, inplace=True)
df['acquisition_cost_gs'].fillna(0, inplace=True)

In [6]:
df_no_operation = df[df['operation_name'].isnull()]
df_on_operation_1 = df[df['operation_name'].notnull()]

In [8]:
#df_on_operation_1 =  df_on_operation_1.loc[df_on_operation_1['Category_1'] == 'BAGAGES']

# Créer une nouvelle ligne avec la valeur "Corner" dans la colonne "Category_1"
new_row = {
    'product_id_upper': 'BEK8690842527838',
    'product_id': 'BEK8690842527838',
    'order_date': '2024-04-07T00:00:00.000+00:00',
    'on_operation': 1,
    'on_google_shopping': 0,
    'type': '1P',
    'avg_price': 499.99,
    'indice_avg_price': 98.23,
    'nb_new_customers': 10,
    'total_customers': 10,
    'operation_name': 'null',
    'startdate_op': 'null',
    'enddate_op': 'null',
    'display_date_gs': 'null',
    'impression_gs': 0,
    'acquisition_cost_gs': 0,
    'tracking_day_front': 'null',
    'on_front': 0,
    'click_on_front': 'null',
    'category': 'ELECTROMENAGER',
    'product_id_1': 'BEK8690842527838',
    'Category_1': 'ELECTROMENAGER',
    'day_of_week_number': 6,
    'week_in_month': 1
}

# Ajouter la nouvelle ligne au DataFrame
df_on_operation_1 = df_on_operation_1.append(new_row, ignore_index=True)

# Modifier la valeur de la colonne "Category_1" pour la nouvelle ligne ajoutée
df_on_operation_1.loc[df_on_operation_1['product_id_upper'] == 'BEK8690842527838', 'Category_1'] = 'CORNER'

# Afficher le DataFrame mis à jour
df_on_operation_1

/var/folders/hm/xzr5dzsn36x5q3_9gvhtjqm00000gn/T/ipykernel_46066/2326120452.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_on_operation_1 = df_on_operation_1.append(new_row, ignore_index=True)


,product_id_upper,product_id,order_date,on_operation,on_google_shopping,type,avg_price,indice_avg_price,nb_new_customers,total_customers,...,impression_gs,acquisition_cost_gs,tracking_day_front,on_front,click_on_front,category,product_id_1,Category_1,day_of_week_number,week_in_month
0,DODOLETOPAFFAI60,DODOLETOPAFFAI60,2024-02-19 00:00:00,0,0,1P,15.99,77.73,0,80,...,0.0,0.00,NaT,0,NaN,None,DODOLETOPAFFAI60,MAISON,0,3
1,DODOLETOPAFFAI60,DODOLETOPAFFAI60,2024-02-19 00:00:00,0,0,3P,16.99,80.94,0,0,...,0.0,0.00,NaT,0,NaN,None,DODOLETOPAFFAI60,MAISON,0,3
2,5904804905,5904804905,2024-02-19 00:00:00,0,1,1P,139.99,88.15,0,40,...,29215.0,29.10,2024-02-19,1,244.0,jardin,5904804905,JARDIN,0,3
3,CELED32SAHD24B3,CELED32SAHD24B3,2024-02-19 00:00:00,0,1,1P,149.99,94.82,0,120,...,35740.0,65.10,2024-02-19,1,298.0,None,CELED32SAHD24B3,HIGH-TECH,0,3
4,CELED55SGUHD24B6,CELED55SGUHD24B6,2024-02-19 00:00:00,0,1,1P,339.99,89.48,0,25,...,4645.0,3.05,2024-02-19,1,65.0,None,CELED55SGUHD24B6,HIGH-TECH,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48081,OCEALL610WS,OCEALL610WS,2024-04-17 00:00:00,1,1,1P,199.99,77.85,10,130,...,31535.0,64.80,NaT,0,NaN,None,OCEALL610WS,ELECTROMENAGER,2,3
48082,WHIOWFC2C26X,WHIOWFC2C26X,2024-04-17 00:00:00,1,1,1P,339.99,84.46,15,145,...,50420.0,130.80,2024-04-17,1,1.0,maison,WHIOWFC2C26X,MAISON,2,3
48083,NXAWFEF001,NXAWFEF001,2024-04-17 00:00:00,1,1,1P,199.99,84.35,10,75,...,20740.0,51.30,NaT,0,NaN,None,NXAWFEF001,INFORMATIQUE,2,3
48084,BEK8690842527838,BEK8690842527838,2024-04-07T00:00:00.000+00:00,1,0,1P,499.99,98.23,10,10,...,0.0,0.00,null,0,null,ELECTROMENAGER,BEK8690842527838,CORNER,6,1


In [10]:
# Séparation des caractéristiques (X) et de la variable cible (y)

X = df_on_operation_1.drop(columns=["on_operation","product_id","type","order_date","nb_new_customers","total_customers","operation_name","startdate_op","enddate_op","display_date_gs","tracking_day_front","click_on_front","category","on_google_shopping","acquisition_cost_gs","product_id_1","product_id_upper"])
#X = NBA_df[["mp"]]
y = df_on_operation_1["nb_new_customers"]


#Données numériques et non numérique train

numerical_columns = X.select_dtypes(include=np.number).columns

cat_columns = X.select_dtypes(exclude=np.number).columns

X=pd.get_dummies(X,columns=cat_columns)


# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)



In [11]:
X.shape

(48086, 26)

In [12]:
scaler = StandardScaler()

# Scale données numériques

#X_train[numerical_columns]= scaler.fit_transform(X_train[numerical_columns])
#X_test[numerical_columns]= scaler.transform(X_test[numerical_columns])


In [13]:
X_train

,avg_price,indice_avg_price,impression_gs,on_front,day_of_week_number,week_in_month,Category_1_ANIMALERIE,Category_1_AU-QUOTIDIEN,Category_1_AUTO,Category_1_BAGAGES,...,Category_1_JEUX-PC-VIDEO-CONSOLE,Category_1_JUNIORS,Category_1_MAISON,Category_1_MEUBLE,Category_1_PRÊT-À-PORTER,Category_1_PUÉRICULTURE,Category_1_SPORT,Category_1_TELEPHONIE,Category_1_TV SON,Category_1_VIN-CHAMPAGNE
14234,299.99,100.00,90.0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30321,25.99,58.77,225.0,0,1,4,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27216,119.99,92.67,1510.0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46235,209.95,116.67,380.0,0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44412,27.99,56.04,105.0,0,6,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21243,119.90,96.00,9665.0,0,4,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45891,59.99,85.71,340.0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
42613,126.90,79.36,1080.0,0,5,5,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
43567,269.00,74.93,225.0,0,6,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Créer et ajuster un modèle de forêt aléatoire
rf_model_op = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_op.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [15]:
# Prédiction et évaluation
y_pred_rf = rf_model_op.predict(X_test)
print(f'RMSE (Random Forest): {mean_squared_error(y_test, y_pred_rf, squared=False)}')
print(f'R^2 (Random Forest): {r2_score(y_test, y_pred_rf)}')

RMSE (Random Forest): 2.1968249384724854
R^2 (Random Forest): 0.23191116529324862


In [16]:
from sklearn.linear_model import LinearRegression 
lin_reg_op = LinearRegression()


# Ajustement du modèle aux données d'entraînement
lin_reg_op.fit(X_train, y_train)

# Évaluation du modèle sur les données de test
lin_reg_op.score(X_test,y_test)
y_pred = lin_reg_op.predict(X_test)
print(lin_reg_op.score(X_test,y_test))

0.1569674391392376


In [17]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
model_GBC_op = GradientBoostingClassifier()

# Entraîner le modèle
model_GBC_op.fit(X_train, y_train)

# Faire des prédictions
predictions = model_GBC_op.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9636098981077147


In [19]:
data = {
    'avg_price': [199], 
    'indice_avg_price': [183], 
    'impression_gs': [87778], 
    'on_front': [0], 
    'day_of_week_number': [4], 
    'week_in_month': [4], 
    'Category_1_ANIMALERIE': [0], 
    'Category_1_AU-QUOTIDIEN': [0], 
    'Category_1_AUTO': [0], 
    'Category_1_BAGAGES': [0], 
    'Category_1_BRICOLAGE': [0], 
    'Category_1_CORNER': [0], 
    'Category_1_ELECTROMENAGER': [0], 
    'Category_1_HIGH-TECH': [0], 
    'Category_1_INFORMATIQUE': [0], 
    'Category_1_JARDIN': [0], 
    'Category_1_JEUX-PC-VIDEO-CONSOLE': [0], 
    'Category_1_JUNIORS': [0], 
    'Category_1_MAISON': [0], 
    'Category_1_MEUBLE': [0], 
    'Category_1_PRÊT-À-PORTER': [1], 
    'Category_1_PUÉRICULTURE': [0], 
    'Category_1_SPORT': [0], 
    'Category_1_TELEPHONIE': [0], 
    'Category_1_TV SON': [0], 
    'Category_1_VIN-CHAMPAGNE': [0]
}

# Créer le DataFrame à partir du dictionnaire
df = pd.DataFrame(data)

display(df)


,avg_price,indice_avg_price,impression_gs,on_front,day_of_week_number,week_in_month,Category_1_ANIMALERIE,Category_1_AU-QUOTIDIEN,Category_1_AUTO,Category_1_BAGAGES,...,Category_1_JEUX-PC-VIDEO-CONSOLE,Category_1_JUNIORS,Category_1_MAISON,Category_1_MEUBLE,Category_1_PRÊT-À-PORTER,Category_1_PUÉRICULTURE,Category_1_SPORT,Category_1_TELEPHONIE,Category_1_TV SON,Category_1_VIN-CHAMPAGNE
0,199,183,87778,0,4,4,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [20]:
print(rf_model_op.predict(df))
print(lin_reg_op.predict(df))
print(model_GBC_op.predict(df))

[3.76666667]
[5.11170278]
[20.]


In [23]:
with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_streamlit_ml/data/fitted_model_rf_op.pickle','wb') as modelFile:
     pickle.dump(rf_model_op,modelFile)

with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_streamlit_ml/data/fitted_model_lr_op.pickle','wb') as modelFile:
     pickle.dump(lin_reg_op,modelFile)

with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_streamlit_ml/data/fitted_model_gbc_op.pickle','wb') as modelFile:
     pickle.dump(model_GBC_op,modelFile)